![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Crimes in Boston

Times, Locations and descriptions of crimes

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)

* [2. MLC2: Data understanding](#data_understanding)
    * [2.1. MLC2.1.: Univariate data analysis](#univariate_data_analysis)
        * [2.1.1 - 2.1.2. Dataset size and direct visualization of the data](#dataset_size_and_visualization)
        * [2.1.3. Types of variables available](#variable_types)
        * [2.1.4. - 2.1.5. Descriptive statistics and number of null values](#descriptive_statistics_and_nulls)
        * [2.1.6. Distribution of target](#target_distribution)
        * [2.1.7. Correlation between variables and target](#outliers_identification)
        * [2.1.8. Identification of outliers](#outliers_identification)
        * [2.1.9. Identification of errors in the data](#errors_data)
        * [2.1.10. Visualization of distributions](#data_distributions)
    * [2.2. MLC2.2.: Multivariate data analysis](#multivariate_data_analysis)
        
* [3. MLC3: Data preparation](#data_preparation)
    * [3.1. MLC 3.1. Data cleaning](#data_cleaning)
        * [3.1.1. Imputation of null values](#nulls_imputation)            
        * [3.1.2. Handling outliers](#handling_outliers)
        * [3.1.3. Dealing with variable types](#dealing_variable_types)
        * [3.1.4. Elimination of features with low variance or highly correlated with others](#low_variance)
    * [3.2. MLC 3.2. Data transformation](#data_transformation)
        * [3.2.1. Transformation of categorical variables](#transformation_categorical)
        * [3.2.2. Transformation of numerical variables](#transformation_numerical)
    * [3.3. MLC 3.3. Feature engineering](#feature_engineering)

Note: Take into account that the ML checklist has been also adapted to the needs of the specific dataset. Some sections may not follow the expected order or some operations may have been done in an earlier stage after confirming these were possible.

## A. Introduction <a class="anchor" id="introduction"></a>

We are working with the BPD (Boston Police Department) with the task to helping to derive some insights from the crime data they provided. Our work is to include all stages of the ML Checklist (data understanding, data preparation, graphs and clusters) and create a clear and defined story over it.

## B. Importing Libraries <a class="anchor" id="libraries"></a>

In [91]:
# system and url requests
import os.path
import requests

# data manipulation 
import pandas as pd
import numpy as np

## C. Importing data<a class="anchor" id="data"></a>

In [92]:
# download Google Drive files helper functions
# useful as it doesn't use any Google third party libs, as we only want to download the data
# Adapted from https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/39225039#39225039

def get_file_id(gdrive_url):
    """
    Extracts the file ID from a Google Drive URL
    """
    return gdrive_url.split('/')[-2]

def get_confirm_token(response):
    """
    Gets the randomly generated confirm token by Google Drive
    """
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    """
    Saves file contents from the response in the given destination
    """
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_file_from_google_drive(gdrive_url, destination):
    """
    Downloads and stores the file from the given url in the destination folder
    """
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    id = get_file_id(gdrive_url)
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)

def download_files(files_list, destination_directory):
    """
    Downloads a list of files from a 'file_list' dictionary into a 'destination_directory'.

    'file_list' is a list of dictionaries with the follwoing mandatory keys:
        - 'name': Name of the file
        - 'url': URL from which the file should be downloaded

    Returns a list of directories where the downloaded files can be found
    """
    dir_list = []
    for file in files_list:
        # check if destination_folder exists and creates it otherwise
        if not os.path.exists(destination_directory):
            print("Creating", destination_directory, "directory", end = '\n\n')
            os.makedirs(destination_directory)

        file_path = './' + destination_directory + '/' + file['name']
        if not os.path.isfile(file_path):
            print(file['name'])
            print("Downloading from:", file['url'], end = '\n\n')
            download_file_from_google_drive(file['url'], file_path)
        else:
            print(file['name'], "already inside of", destination_directory)
        
        dir_list.append(file_path)
    return dir_list

In [93]:
# downloading the files from Google Drive URLs
directory = 'data/original'

files_to_download = [
  {
    'name': 'crime.csv',
    'url': 'https://drive.google.com/file/d/11y05JjYleaA0FRB4448GH5EKka-1C26Z/view?usp=sharing'
  },
  {
    'name': 'offense_codes.csv',
    'url': 'https://drive.google.com/file/d/122eCg0ieLEAEY95mKuwRjJb0xztBalW_/view?usp=sharing'
  }
]

file_dirs = download_files(files_to_download, directory)

crime.csv already inside of data/original
offense_codes.csv already inside of data/original


In [94]:
# importing data into Pandas DataFrames
crimes = pd.read_csv(file_dirs[0], encoding = 'windows-1252')
oc = pd.read_csv(file_dirs[1], encoding = 'windows-1252')

## MLC2: Data Understanding<a class="anchor" id="data_understanding"></a>

### MLC2.1: Univariate data analysis<a class="anchor" id="univariate_data_analysis"></a>

First of all, let's review the 2 datasets we generated `crimes` and `oc` to see with what type of data we are dealing with.

### 2.1.1 - 2.1.2. Dataset size and direct visualization of the data<a class="anchor" id="dataset_size_and_visualization"></a>

#### `crimes`

In [95]:
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [96]:
crimes.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')

Each row in the `crimes` dataset contains a crime report, including the type, the district and street where it happened, day and time...

In [97]:
print("There are", crimes.shape[0], "records, and", crimes.shape[1], "columns to review")

There are 319073 records, and 17 columns to review


In [98]:
print("Crime records go from", crimes.OCCURRED_ON_DATE.min(), "until", crimes.OCCURRED_ON_DATE.max())

Crime records go from 2015-06-15 00:00:00 until 2018-09-03 21:25:00


In [99]:
crimes.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319073 entries, 0 to 319072
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   INCIDENT_NUMBER      319073 non-null  object 
 1   OFFENSE_CODE         319073 non-null  int64  
 2   OFFENSE_CODE_GROUP   319073 non-null  object 
 3   OFFENSE_DESCRIPTION  319073 non-null  object 
 4   DISTRICT             317308 non-null  object 
 5   REPORTING_AREA       319073 non-null  object 
 6   SHOOTING             1019 non-null    object 
 7   OCCURRED_ON_DATE     319073 non-null  object 
 8   YEAR                 319073 non-null  int64  
 9   MONTH                319073 non-null  int64  
 10  DAY_OF_WEEK          319073 non-null  object 
 11  HOUR                 319073 non-null  int64  
 12  UCR_PART             318983 non-null  object 
 13  STREET               308202 non-null  object 
 14  Lat                  299074 non-null  float64
 15  Long             

#### `oc` (Offense Codes)

In [100]:
oc.head()

,CODE,NAME
0,612,LARCENY PURSE SNATCH - NO FORCE
1,613,LARCENY SHOPLIFTING
2,615,LARCENY THEFT OF MV PARTS & ACCESSORIES
3,1731,INCEST
4,3111,LICENSE PREMISE VIOLATION


In [101]:
oc.columns

Index(['CODE', 'NAME'], dtype='object')

Each row in the `oc` DataFrame corresponds to an Offense Code Number and its Name (or description). This may be useful in order to understand what does the `OFFENSE_CODE` mean in the `crimes` dataset.

In [102]:
print("There are", oc.shape[0], "records, and", oc.shape[1], "columns to review")

There are 576 records, and 2 columns to review


In [103]:
oc.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODE    576 non-null    int64 
 1   NAME    576 non-null    object
dtypes: int64(1), object(1)
memory usage: 9.1+ KB


### 2.1.3. Types of variables available<a class="anchor" id="variable_types"></a> 

#### Merging `oc` with `crimes`

Before starting with any analysis on the crimes dataset, we will try to merge the `oc` data with the `crimes` dataset, as it seems that, when inspecting the data in `crimes` there may be some columns with repeated information that we could try removing (mainly `OFFENSE_CODE`, `OFFENSE_CODE_GROUP`, `OFFENSE_DESCRIPTION`).

In [104]:
crimes.OFFENSE_CODE.nunique()

222

In [105]:
oc.CODE.nunique()

425

It seems that we have more Offense Codes in `oc` than in crimes. However, this may mean that we have unused values in `crimes`, which is fine. Nonetheless, what is kind of worrying is that there are 425 unique codes in `oc`, but we have 576 records, so it means that we have some duplicates. Let's check them out.

In [106]:
oc.sort_values('CODE').head(20)

,CODE,NAME
183,111,MURDER NON-NEGLIGIENT MANSLAUGHTER
19,111,"MURDER, NON-NEGLIGIENT MANSLAUGHTER"
154,112,KILLING OF FELON BY POLICE
184,112,KILLING OF FELON BY POLICE
542,113,KILLING OF FELON BY CITIZEN
185,114,KILLING OF POLICE BY FELON
155,114,KILLING OF POLICE BY FELON
186,121,MANSLAUGHTER - VEHICLE - NEGLIGENCE
15,121,MANSLAUGHTER - VEHICLE - NEGLIGENCE
187,122,MANSLAUGHTER - TRAIN ETC. VICTIM NON-NEGLIGENCE


By inspecting the data we see, for example that for `CODE = 111` we have 2 entries which mean the same and it only changes a comma. For `CODE = 112` we have basically the same info. So we'll drop duplicates and we'll keep the first entry we encounter. Additionally, we have seen that there may be values with commas (`CODE = 111`), this is something to take into account for as soon as we merge both DataFrames and checking the result.

In [107]:
# removing commas, so we are getting a standardized NAME column
oc.NAME = oc.NAME.str.replace(',', '')

In [108]:
# dealing with duplicate codes
oc.drop_duplicates(subset = 'CODE', keep = 'first', inplace = True)

In [109]:
oc.shape

(425, 2)

Now we can merge `oc` code names with `crimes`

In [110]:
oc.columns = ['OFFENSE_CODE', 'OFFENSE_NAME']
crimes = crimes.merge(oc, on='OFFENSE_CODE', how='inner')

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,OFFENSE_NAME
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",LARCENY ALL OTHERS
1,I182070885,619,Larceny,LARCENY ALL OTHERS,B3,456,NaN,2018-09-03 17:17:00,2018,9,Monday,17,Part One,ELLINGTON ST,42.301546,-71.081182,"(42.30154555, -71.08118184)",LARCENY ALL OTHERS
2,I182070816,619,Larceny,LARCENY ALL OTHERS,C6,185,NaN,2018-09-03 12:45:00,2018,9,Monday,12,Part One,MASSACHUSETTS AVE,42.321987,-71.062807,"(42.32198656, -71.06280666)",LARCENY ALL OTHERS
3,I182070777,619,Larceny,LARCENY ALL OTHERS,C11,388,NaN,2018-08-23 13:34:00,2018,8,Thursday,13,Part One,ADAMS ST,42.272306,-71.067214,"(42.27230624, -71.06721386)",LARCENY ALL OTHERS
4,I182070707,619,Larceny,LARCENY ALL OTHERS,D14,782,NaN,2018-09-02 23:12:00,2018,9,Sunday,23,Part One,GORDON ST,42.352935,-71.140501,"(42.35293536, -71.14050052)",LARCENY ALL OTHERS
5,I182070696,619,Larceny,LARCENY ALL OTHERS,D4,623,NaN,2018-09-02 22:15:00,2018,9,Sunday,22,Part One,COMMONWEALTH AVE,42.348776,-71.096068,"(42.34877575, -71.09606805)",LARCENY ALL OTHERS
6,I182070607,619,Larceny,LARCENY ALL OTHERS,E18,519,NaN,2018-08-24 00:00:00,2018,8,Friday,0,Part One,WALTER ST,42.256679,-71.118540,"(42.25667872, -71.11853971)",LARCENY ALL OTHERS
7,I182070596,619,Larceny,LARCENY ALL OTHERS,C6,216,NaN,2018-09-02 17:29:00,2018,9,Sunday,17,Part One,F ST,42.333829,-71.051287,"(42.33382906, -71.05128729)",LARCENY ALL OTHERS
8,I182070525,619,Larceny,LARCENY ALL OTHERS,C11,397,NaN,2018-09-02 11:45:00,2018,9,Sunday,11,Part One,TALBOT AVE,42.288638,-71.066316,"(42.28863812, -71.06631579)",LARCENY ALL OTHERS
9,I182070499,619,Larceny,LARCENY ALL OTHERS,C6,186,NaN,2018-09-02 09:17:00,2018,9,Sunday,9,Part One,ALLSTATE RD,42.328100,-71.063217,"(42.32809966, -71.06321676)",LARCENY ALL OTHERS


By simple inspection, we can see that maybe `OFFENSE_DESCRIPTION` and `OFFENSE_NAME` are the same, so let's double check.

In [87]:
crimes.OFFENSE_DESCRIPTION.nunique()

244

In [88]:
crimes.OFFENSE_NAME.nunique()

221

There are less unique values in the newly created column than in the old one. Checking if this is due to null values:

In [89]:
crimes.isna().sum()

INCIDENT_NUMBER             0
OFFENSE_CODE                0
OFFENSE_CODE_GROUP          0
OFFENSE_DESCRIPTION         0
DISTRICT                 1765
REPORTING_AREA              0
SHOOTING               318054
OCCURRED_ON_DATE            0
YEAR                        0
MONTH                       0
DAY_OF_WEEK                 0
HOUR                        0
UCR_PART                   90
STREET                  10871
Lat                     19999
Long                    19999
Location                    0
OFFENSE_NAME                0
dtype: int64

That's not the case, there are no null values in these columns... so let's inspect the values that are different between these 2 columns.

In [111]:
# removing also the commas from OFFENSE_DESCRIPTION so we can compare the values properly
crimes.OFFENSE_DESCRIPTION = crimes.OFFENSE_DESCRIPTION.str.replace(',', '')

In [112]:
crimes.loc[crimes['OFFENSE_NAME'] != crimes['OFFENSE_DESCRIPTION'], ['OFFENSE_DESCRIPTION', 'OFFENSE_NAME']].drop_duplicates()

,OFFENSE_DESCRIPTION,OFFENSE_NAME
5962,LARCENY OTHER $200 & OVER,LARCENY ALL OTHERS
112135,LARCENY NON-ACCESSORY FROM VEH. $200 & OVER,LARCENY THEFT FROM MV - NON-ACCESSORY
122550,B&E RESIDENCE DAY - NO FORCE,BURGLARY - RESIDENTIAL - NO FORCE
153661,ASSAULT & BATTERY,ASSAULT SIMPLE - BATTERY
177155,LARCENY SHOPLIFTING $200 & OVER,LARCENY SHOPLIFTING
179845,DRUGS - POSS CLASS B - INTENT TO MFR DIST DISP,DRUGS - PRESENT AT HEROIN
191690,M/V ACCIDENT - PROPERTY DAMAGE,M/V ACCIDENT - PROPERTY ÊDAMAGE
201049,DISORDERLY PERSON,DISORDERLY CONDUCT
210116,LARCENY IN A BUILDING $200 & OVER,LARCENY THEFT FROM BUILDING
236128,LARCENY BICYCLE $200 & OVER,LARCENY THEFT OF BICYCLE


Seems that in the fields where there are differences, these are very similar between them, so... why is that? Let's do some more analysis to see if there are any repeated values in `OFFENSE_DESCRIPTION`.

In [130]:
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY ALL OTHERS', ['OFFENSE_DESCRIPTION']].count())
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY OTHER $200 & OVER', ['OFFENSE_DESCRIPTION']].count())

OFFENSE_DESCRIPTION    5963
dtype: int64
OFFENSE_DESCRIPTION    7
dtype: int64


In [131]:
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY NON-ACCESSORY FROM VEH. $200 & OVER', ['OFFENSE_DESCRIPTION']].count())
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY THEFT FROM MV - NON-ACCESSORY', ['OFFENSE_DESCRIPTION']].count())

OFFENSE_DESCRIPTION    1
dtype: int64
OFFENSE_DESCRIPTION    8893
dtype: int64


By looking at a couple of the fields, we see that `OFFENSE_DESCRIPTION` has not only some fields with the same values as in `OFFENSE_NAME` but also some other fields which are causing the differences. So, in order to keep our data the most standardized possible, we will remove the `OFFENSE_DESCRIPTION` column and keep the one we generated from the `oc` DataFrame.

In [132]:
crimes.drop(columns = 'OFFENSE_DESCRIPTION', axis = 1, inplace = True)